In [40]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
import datetime

In [2]:
df = pd.read_csv('Urbana_Crime_Data.csv', low_memory=False)

cols = ['INCIDENT', 'DATE OCCURRED', 'TIME OCCURRED', 'YEAR OCCURRED', 'MONTH OCCURRED', 'GEO CODE', 'HOUSE NUMBER BLOCK', 'STREET', 'CRIME CODE', 'CRIME DESCRIPTION', 'CRIME CATEGORY', 'MAPPING ADDRESS']

print(df.columns)

crimes = df[cols]

print(crimes.columns)

crimes.head()

Index(['INCIDENT', 'DATE OCCURRED', 'TIME OCCURRED', 'YEAR OCCURRED',
       'MONTH OCCURRED', 'DATE REPORTED', 'TIME REPORTED', 'DATE ARRIVED',
       'TIME ARRIVED', 'GEO CODE', 'HOUSE NUMBER BLOCK', 'STREET',
       'CROSS STREET', 'CRIME CODE', 'CRIME DESCRIPTION', 'CRIME CATEGORY',
       'CRIME CATEGORY DESCRIPTION', 'CSA DESCRIPTION', 'PLACE CODE',
       'PLACE CODE DESCRIPTION', 'WEAPONS CODE 1', 'WEAPON 1 DESCRIPTION',
       'WEAPONS CODE 2', 'WEAPON 2 DESCRIPTION', 'WEAPONS CODE 3',
       'WEAPON 3 DESCRIPTION', 'BIAS CODE', 'BIAS DESCRIPTION', 'STATUS CODE',
       'STATUS DESCRIPTION', 'COUNTER', 'MAPPING ADDRESS'],
      dtype='object')
Index(['INCIDENT', 'DATE OCCURRED', 'TIME OCCURRED', 'YEAR OCCURRED',
       'MONTH OCCURRED', 'GEO CODE', 'HOUSE NUMBER BLOCK', 'STREET',
       'CRIME CODE', 'CRIME DESCRIPTION', 'CRIME CATEGORY', 'MAPPING ADDRESS'],
      dtype='object')


,INCIDENT,DATE OCCURRED,TIME OCCURRED,YEAR OCCURRED,MONTH OCCURRED,GEO CODE,HOUSE NUMBER BLOCK,STREET,CRIME CODE,CRIME DESCRIPTION,CRIME CATEGORY,MAPPING ADDRESS
0,U19-01720,04/13/2019,17:20,2019,4,605-04,1100 BLOCK,COLORADO AV,1310.0,CRIMINAL DAMAGE TO PROPERTY,C17,"1100 COLORADO AV\nURBANA, IL\n(40.094723, -88...."
1,U19-00373,01/24/2019,16:13,2019,1,546-01,1200 BLOCK,EUREKA ST,486.0,BATTERY-DOMESTIC,C05,"1200 EUREKA ST\nURBANA, IL\n(40.125577, -88.22..."
2,U19-02382,05/21/2019,12:00,2019,5,576-06,600 BLOCK,VINE ST S,1110.0,DECEPTIVE PRACTICES,C16,"600 VINE ST S\nURBANA, IL\n(40.108475, -88.204..."
3,U19-04319,08/28/2019,07:45,2019,8,547-02,1300 BLOCK,CUNNINGHAM AVE,460.0,BATTERY,C05,"1300 CUNNINGHAM AVE\nURBANA, IL\n(40.125562, -..."
4,U19-00422,01/27/2019,10:45,2019,1,605-04,1200 BLOCK,FLORIDA AV E,1365.0,TRESPASS-RESIDENCE,C77,"1200 FLORIDA AV E\nURBANA, IL\n(40.098384, -88..."


In [3]:
## Mapping specific crimes to broader catogories

crime_category_cols = ['CRIME CATEGORY', 'CRIME DESCRIPTION']

crime_category_unique = crimes['CRIME CATEGORY'].drop_duplicates(keep='last')

crime_category_df = df[crime_category_cols].drop_duplicates(keep='last')
crime_category_df = crime_category_df.set_index('CRIME DESCRIPTION')

print(crime_category_df.columns)

crime_category_df.head(20)

# print(crime_category_df.index[0])

# print(crime_category_df.loc[crime_category_df.index[0] , 'CRIME CATEGORY'])

Index(['CRIME CATEGORY'], dtype='object')


,CRIME CATEGORY
CRIME DESCRIPTION,
METHAMPHETAMINE - MANUFACTURING,C23
TAXI PROVISIONAL LICENSE,C62
IMPROPER DEPOSIT OF GARBAGE,C47
PARKING-2 AM TO 6 AM,C28
PROSTITUTION,C19
UNLAWFUL USE OF A BUILDING,C24
IMPROPER U-TURN,C28
INTIMIDATION-AGGRAVATED,C32
GRADUATED DRIVER,C28


In [4]:
all_crime_categories = {}

first_index = crime_category_unique.first_valid_index()
last_index = crime_category_unique.last_valid_index()

indices = crime_category_df.index

for category in crime_category_unique:

    arr = []

    all_crime_categories.update({category : []})

    for i in range(len(crime_category_df)):
        curr = crime_category_df.loc[crime_category_df.index[i] , 'CRIME CATEGORY']

        if (curr is category):
            arr.append(crime_category_df.index[i])

    all_crime_categories[category] = arr

print(all_crime_categories)

{'C79': ['USE/SALE/DEL OF INTOXICATING COMPOU'], 'C84': ['TERRY STOPS'], 'C63': ['BURGLAR ALARM'], 'C43': ['PEDESTRIAN WALKING ON ROADWAY'], 'C75': ['ENDANGERMENT'], 'C59': ['PARKING ENFORCEMENT BOOT'], 'C57': ['OPEN BURNING'], 'C36': ['OTHER/MISC CITY ORDINANCE VIOLATION', 'OTHER CRIMINAL OFFENSES'], 'C65': ['BURGLARY TOOLS-POSSESSION'], 'C44': ['ALL OTHER PARK DIST VIOLATIONS'], 'C34': ['TERRORISM - MAKE THREATS'], 'C25': ['HYPODERMIC NEEDLE/SYRINGE-POSS/SALE'], 'C01': ['HOMICIDE-DRUG INDUCED', 'MURDER-FIRST DEGREE'], 'C82': ['OTHER PUBLIC COMPLAINTS'], 'C53': ['PARKING-HANDICAP', 'PARKING VIOLATIONS-ALL OTHER', 'PARKING VIOLATIONS-ILLEGAL PARKING'], 'C22': ['CANNABIS TRAFFICKING', 'POSS OF CANNABIS', 'UNLAWFUL CANNABIS MANUFACTURING', 'CANNABIS POSS OVER 100 G -FEL', 'DEL/MANUF CANNABIS UNDER 10 G -MISD', 'CANNABIS POSS OVER 10 - 100 G -MISD', 'CANNABIS POSS 10 GM OR LESS -ORD', 'DEL/MANUF CANNABIS OVER 30 G -FEL'], 'C32': ['INTIMIDATION-AGGRAVATED', 'CYBERSTALKING', 'STALKING', 'IN

In [5]:
scores = {'CO1' : 15, 'C02' : 12, 'C08' : 12, 'C03' : 10, 'C05' : 10, 'C09' : 10, 'C34' : 10, 'C10' : 8, 'C11' : 8, 'C28' : 8, 'C27' : 6}

In [6]:
crime_categories = {}

for crime in all_crime_categories.keys():
    if crime in scores:
        crime_categories.update({crime : all_crime_categories[crime]})

In [7]:
filtered_crimes = crimes

for c in range(crimes.first_valid_index(), crimes.last_valid_index() + 1):
    if (crimes.loc[c])['CRIME CATEGORY'] not in scores:
        filtered_crimes = filtered_crimes.drop(c)

# filtered_crimes = filtered_crimes.set_index('CRIME DESCRIPTION')

filtered_crimes.head()

,INCIDENT,DATE OCCURRED,TIME OCCURRED,YEAR OCCURRED,MONTH OCCURRED,GEO CODE,HOUSE NUMBER BLOCK,STREET,CRIME CODE,CRIME DESCRIPTION,CRIME CATEGORY,MAPPING ADDRESS
1,U19-00373,01/24/2019,16:13,2019,1,546-01,1200 BLOCK,EUREKA ST,486.0,BATTERY-DOMESTIC,C05,"1200 EUREKA ST\nURBANA, IL\n(40.125577, -88.22..."
3,U19-04319,08/28/2019,07:45,2019,8,547-02,1300 BLOCK,CUNNINGHAM AVE,460.0,BATTERY,C05,"1300 CUNNINGHAM AVE\nURBANA, IL\n(40.125562, -..."
7,U19-02433,05/23/2019,15:14,2019,5,578-02,100 BLOCK,HIGH CROSS RD S,6621.0,FAILURE TO REDUCE SPEED,C28,"100 HIGH CROSS RD S\nURBANA, IL\n(40.111989, -..."
8,T19-01952,04/28/2019,10:01,2019,4,605-13,800 BLOCK,WINDSOR RD E,6601.0,SPEEDING (RADAR),C28,"800 WINDSOR RD E\nURBANA, IL\n(40.083797, -88...."
14,U19-01353,03/23/2019,18:15,2019,3,606-03,2500 BLOCK,PHILO RD,8612.0,THEFT-RETAIL-CITY,C11,"2500 PHILO RD\nURBANA, IL\n(40.085441, -88.190..."


In [8]:
timesData = pd.DataFrame(filtered_crimes)

timesList = timesData['TIME OCCURRED'].tolist()

print(timesList)

['16:13', '07:45', '15:14', '10:01', '18:15', '18:25', '15:00', '08:27', '08:20', '23:00', '10:10', '23:16', '19:40', '23:30', '10:53', '15:08', '15:27', '09:46', '14:04', '10:55', '08:32', '22:00', '17:00', '17:44', '18:15', '07:17', '03:44', '20:53', '20:00', '14:25', '21:05', '01:05', '22:00', '19:01', '10:45', '16:00', '10:22', '20:45', '15:00', '17:00', '17:41', '10:22', '00:01', '18:40', '16:15', '19:52', '19:21', '02:22', '14:10', '12:00', '09:30', '14:45', '00:01', '10:51', '13:10', '23:43', '15:27', '21:00', '17:15', '16:38', '13:00', '21:40', '02:30', '20:45', '14:13', '13:01', '08:12', '22:45', '22:15', '04:57', '09:30', '13:20', '22:54', '18:00', '12:06', '23:24', '11:00', '05:00', '01:30', '08:23', '13:50', '10:50', '22:00', '13:45', '19:00', '15:15', '21:45', '15:00', '16:42', '18:34', '21:15', '19:17', '01:00', '04:59', '19:45', '01:57', '18:00', '12:48', '17:03', '18:00', '12:00', '15:56', '15:50', '14:20', '08:03', '03:11', '19:00', '23:14', '17:50', '16:40', '23:25', 

In [9]:
counts = [0] * 24
total = 0

for time in timesList:
    if (int(time[0:2]) > 23):
        continue
    elif (time[:1] == '0'):
        index = int(time[1:2])
    else:
        index = int(time[0:2])
    total += 1
    counts[index] = counts[index] + 1

In [10]:
def get_count(time):
    if (len(time) == 4):
        index = time[0:1]
    elif (len(time) == 5):
        index = time[0:2]
    return counts[int(index)]

print("# of crimes in this hour: " + str(get_count("4:30")))

# of crimes in this hour: 130


In [39]:
def get_multiplier(time):
    if (len(time) == 4):
        index = time[0:1]
    elif (len(time) == 5):
        index = time[0:2]
    return counts[int(index)] / total

print("Score multiplier for this hour: " + str(get_multiplier("4:30")))

Score multiplier for this hour: 0.016581632653061226


In [44]:
def get_current_multiplier():
    current_time = datetime.datetime.now()
    index = int(current_time.hour)
    return counts[int(index)] / total

print("Score multiplier for the current hour: " + str(get_current_multiplier()))

Score multiplier for the current hour: 0.05765306122448979
